In [15]:
import json
with open("/home/work/deeptext/yys/redpen/results/result_readability_newed.json", "r") as f:

    data = json.load(f)

In [16]:
import re

def is_starts_with_num(s):
    regex_start_with_1_to_9 = r'^[1-9]'
    if re.match(regex_start_with_1_to_9,s) is not None:
        return s
    else:
        return False
def is_ends_with_num(string):
    regex_end_with_digit = r'\d$'
    if re.search(regex_end_with_digit, string):
        return string
    else:
        return False

def process_score(score_list):
    pattern = r"[^.\d]"
    new_scores = []
    for i in score_list:
        if str(type(i))=="<class 'int'>" or str(type(i))=="<class 'float'>":
            continue
        else:
            if len(i)>0:
                tmp = re.sub(pattern, "", i)
                if len(tmp)>0 and tmp!="." and is_starts_with_num(tmp) is not False and is_ends_with_num(tmp) is not False and float(tmp)<=5:
                    new_scores.append(float(tmp))
    if len(new_scores)>0:
        return new_scores
    else:
        return []

In [17]:
process_score(['4.5', '3.5', '4.5', '4.5', '4.5'])
pattern = r"[^.\d]"
new_scores = []

for i in ['4.5', '3.5', '4.5', '4.5', '4.5']:
    tmp = re.sub(pattern,"",i)
    if len(tmp)>0 and tmp!="." and is_starts_with_num(tmp) is not False and is_ends_with_num(tmp) is not False and float(tmp)<=5:
        new_scores.append(float(tmp))
new_scores

[4.5, 3.5, 4.5, 4.5, 4.5]

In [4]:
data[0]['gpt-3.5']

{'response': 'The American state that is nicknamed the Treasure State is Montana.',
 'grammar': [5.0, 5.0, 5.0, 5.0, 5.0],
 'completeness': [5.0, 5.0, 5.0, 5.0, 5.0],
 'readability': ['4', '35', '4', '4.5', '4'],
 'grammar_avg': 5.0,
 'completeness_avg': 5.0,
 'readability_avg': 5.0}

In [18]:
process_score(['4', '35', '4', '4.5', '4'])

[4.0, 4.0, 4.5, 4.0]

In [19]:
for datum in data:
    try: 
        datum['gpt-3.5']['readability'] = process_score(datum['gpt-3.5']['readability'])
    except:
        print(datum['gpt-3.5']['readability'])

In [26]:
models = ['gpt-4','llama2','gpt-3.5','dolly_v2']
critics = ['grammar','completeness','readability']
new_list = []
for datum in data:
        new_dict = {'idx':datum['idx'],
                'question':datum['question'],
                'q_len':datum['q_len'],
                'gpt-4':{'response':datum['gpt-4']['response'],
                        'grammar':process_score(datum['gpt-4']['grammar']),
                        'completeness':process_score(datum['gpt-4']['completeness']),
                        'readability':process_score(datum['gpt-4']['readability'])
                        },
                'gpt-3.5':{'response':datum['gpt-3.5']['response'],
                        'grammar':process_score(datum['gpt-3.5']['grammar']),
                        'completeness':process_score(datum['gpt-3.5']['completeness']),
                        'readability':process_score(datum['gpt-3.5']['readability'])
                        },
                'llama2':{'response':datum['llama2']['response'],
                        'grammar':process_score(datum['llama2']['grammar']),
                        'completeness':process_score(datum['llama2']['completeness']),
                        'readability':process_score(datum['llama2']['readability'])
                        },
                'dolly_v2':{'response':datum['dolly_v2']['response'],
                        'grammar':process_score(datum['dolly_v2']['grammar']),
                        'completeness':process_score(datum['dolly_v2']['completeness']),
                        'readability':process_score(datum['dolly_v2']['readability'])
                        }}
        new_list.append(new_dict)
    



TypeError: object of type 'float' has no len()

In [20]:
len(data)

7487

In [22]:
models = ['gpt-4','llama2','gpt-3.5','dolly_v2']
critics = ['grammar','completeness','readability']
index_to_remove = []
for datum in data:
    for model in models:
        for critic in critics:
            if len(datum[model][critic])==0:
                index_to_remove.append(datum['idx'])


In [24]:
for i in reversed(index_to_remove):
    data.pop(i)

In [26]:
for i in range(len(data)):
    data[i]['idx'] = i

In [28]:
len(data)

6817

In [29]:
def response_process(text):
    text = re.sub('[^a-zA-Z~!@#$%^&*()_+|<>?:{}.,; ]', '', text)
    if len(text.split())<3:
        print(text)
        return False
    else:
        return True
response_process('hi, my name is yys.')

True

In [30]:
models = ['gpt-4','llama2','gpt-3.5','dolly_v2']

index_to_remove = []

for datum in data:
    for model in models:
        text = datum[model]['response']
        if not response_process(text):
            index_to_remove.append(datum['idx'])          

In [31]:
index_to_remove = list(set(index_to_remove))
len(index_to_remove)

0

In [13]:
index_to_remove.sort()
for i in reversed(index_to_remove):
    new_list.pop(i)
for i in range(len(new_list)):
    new_list[i]['idx'] = i
len(new_list)

7531

In [32]:
import numpy as np
for datum in data:
    for model in models:
        for critic in critics:
            new_name = critic+'_avg'
            datum[model][new_name] = np.mean(datum[model][critic])
            #datum[model][new_name] = np.mean(sorted(datum[model][critic][:3],reverse=True))

In [15]:
from transformers import AutoModel, AutoTokenizer
model = AutoModel.from_pretrained("microsoft/deberta-v3-large")
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')

/home/work/.local/miniconda3/envs/yys/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/work/.local/miniconda3/envs/yys/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddi

In [16]:
def calc_len(text):
    l = tokenizer.tokenize(text)
    if len(l)>256:
        return False
    else:
        return True

In [17]:
index_to_remove = []
for datum in new_list:
    for model in models:
        if not calc_len(datum[model]['response']):
            index_to_remove.append(datum['idx'])

In [18]:
index_to_remove = list(set(index_to_remove))
index_to_remove.sort()

In [19]:
for i in reversed(index_to_remove):
    new_list.pop(i)
for i in range(len(new_list)):
    new_list[i]['idx'] = i
len(new_list)

7487

In [20]:
with open("/home/work/deeptext/yys/redpen/data/data_final_g3.json", "w", encoding='utf-8') as f:
    json.dump(data,f , ensure_ascii=False,indent=4)